In [1]:
! pip install gradio

In [2]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image

2024-12-01 18:06:13.794792: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-01 18:06:14.236639: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-01 18:06:15.500827: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="model_unquant.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Define the actual class labels
class_names = [
    "Big Cola",
    "V7",
    "BigChips Salt & Vinegar",
    "Toren Wafers"
]

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
# Define the prediction function
def predict_image(image):
    # Resize and preprocess the image to match the model's input size
    image = image.resize((input_details[0]['shape'][1], input_details[0]['shape'][2]))
    input_data = np.expand_dims(image, axis=0).astype(np.float32) / 255.0  # Normalize pixel values

    # Perform inference
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])[0]  # Get the first batch output

    # Find the predicted class and confidence
    predicted_class_idx = np.argmax(output_data)
    predicted_class = class_names[predicted_class_idx]
    confidence_scores = {class_names[i]: float(output_data[i]) for i in range(len(class_names))}

    return predicted_class, confidence_scores


In [5]:
# Create the Gradio interface
iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=[
        gr.Label(label="Predicted Class"),
        gr.Label(label="Confidence Scores"),
    ],
    title="Product Detection with TFLite",
    description="Upload an image to predict the product class using a TFLite model.",
    theme="huggingface",  # Optional: Change theme if desired
)

iface.launch()

/home/abuhamida/.local/lib/python3.10/site-packages/gradio/blocks.py:983: UserWarning: Cannot load huggingface. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/huggingface (Request ID: Root=1-674c89d5-2f96b40a79eaca4c22033bfd;60bd9875-3943-4865-871a-7fba0a7f06d0)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 4.36.0, however version 4.44.1 is available, please upgrade.
--------
